In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [17]:
data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)

# len4_ndb is left joined so delete duplicated rows
data=data.drop_duplicates(subset=["id","year","総計"])
# drop never treated group
data=data.dropna(subset=["Approved"])
data=data.dropna(subset=["総計"])
data.head()

C:\Users\hakos\AppData\Local\Temp\ipykernel_2948\4159044249.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("../merged/len4_ndb.csv",encoding="shift-jis",index_col=0)


,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,oral,id_l4,Approved,KEGG,Ingredient,variable,id,revenue
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,5.639285e+06
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.486458e+06
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,0,1145,1985.0,D00141,Indometacin,id1,1145.0,2.052130e+06
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,1.270436e+07
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,0,1145,1985.0,D00141,Indometacin,id1,1145.0,8.081441e+06


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [18]:
data["elasped"]=data["year"]-data["Approved"]
#　不要なところを除外
# data.loc[(data['elasped'] > 10) | (data["elasped"]< -5), 'elasped'] = np.nan

# dummy をとりたいところを分離
dummies=pd.get_dummies(data["elasped"],prefix="elasped")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
# sumをとりたいところを分離
sumcolumns=["総計","revenue"]
sums=data.groupby(["year","薬効分類"])[sumcolumns].sum().reset_index()
# save temporarily
dummy_cols=dummies.columns.to_list()

# sum by generic
sum_generic=data.groupby(["year","薬効分類","後発品区分"])[sumcolumns].sum().reset_index()


# count ver.
# aggregated_df = pd.concat([
#     data.groupby(['year', '薬効分類']).first().reset_index()[['year', '薬効分類']],
#     dummies.groupby([data['year'], data['薬効分類']]).sum().reset_index(drop=True)
# ], axis=1).reset_index(drop=True)
# dummy ver.
aggregated_df = pd.concat([
    data.groupby(['year', '薬効分類']).first().reset_index()[['year', '薬効分類']],
    dummies.groupby([data['year'], data['薬効分類']]).apply(lambda x: (x[dummy_cols] > 0).any().astype(int)).reset_index(drop=True)
], axis=1).reset_index(drop=True)
# # sumをとったところを追加する

aggregated_df[sumcolumns]=sums[sumcolumns]
aggregated_df.rename(columns={"総計":"sum_quantity","revenue":"sum_revenue"},inplace=True)
aggregated_df["mean_price"]=aggregated_df["sum_revenue"]/aggregated_df["sum_quantity"]
aggregated_df["lag_mean_price"]=aggregated_df.groupby(["薬効分類"])["mean_price"].shift(1)
aggregated_df.head()

,year,薬効分類,elasped_m6,elasped_m5,elasped_m4,elasped_m3,elasped_m2,elasped_m1,elasped_0,elasped_1,...,elasped_33,elasped_34,elasped_35,elasped_36,elasped_37,elasped_38,sum_quantity,sum_revenue,mean_price,lag_mean_price
0,2014,114.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3.431556e+09,1.094992e+11,31.909498,NaN
1,2014,121.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4.840017e+07,3.186312e+08,6.583267,NaN
2,2014,123.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.596449e+08,2.033818e+09,12.739638,NaN
3,2014,124.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8.289082e+08,9.731627e+09,11.740296,NaN
4,2014,131.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8.145575e+08,1.297159e+11,159.247093,NaN


In [19]:
sum_generic["後発品区分"]=sum_generic["後発品区分"].astype("int")
sum_generic=sum_generic.pivot_table(index=["year","薬効分類"],columns="後発品区分",values=sumcolumns,aggfunc="sum").reset_index()
sum_generic.columns = [f"{col[0]}{col[1]}" for col in sum_generic.columns]


In [20]:
aggregated_df[sum_generic.columns]=sum_generic[sum_generic.columns]

In [21]:
aggregated_df

,year,薬効分類,elasped_m6,elasped_m5,elasped_m4,elasped_m3,elasped_m2,elasped_m1,elasped_0,elasped_1,...,elasped_37,elasped_38,sum_quantity,sum_revenue,mean_price,lag_mean_price,revenue0,revenue1,総計0,総計1
0,2014,114.0,0,0,0,0,0,0,0,0,...,0,0,3.431556e+09,1.094992e+11,31.909498,NaN,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08
1,2014,121.0,0,0,0,0,0,0,0,0,...,0,0,4.840017e+07,3.186312e+08,6.583267,NaN,3.016000e+08,1.703117e+07,4.539382e+07,3.006345e+06
2,2014,123.0,0,0,0,0,0,0,0,0,...,0,0,1.596449e+08,2.033818e+09,12.739638,NaN,1.903159e+09,1.306591e+08,1.390288e+08,2.061612e+07
3,2014,124.0,0,0,0,0,0,0,0,0,...,0,0,8.289082e+08,9.731627e+09,11.740296,NaN,7.625584e+09,2.106043e+09,4.668874e+08,3.620207e+08
4,2014,131.0,0,0,0,0,0,0,0,0,...,0,0,8.145575e+08,1.297159e+11,159.247093,NaN,1.192776e+11,1.043834e+10,7.030695e+08,1.114880e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2021,441.0,0,0,0,0,0,0,0,0,...,0,0,1.614168e+08,1.197058e+09,7.415946,7.430559,8.816861e+08,3.153723e+08,1.060883e+08,5.532848e+07
260,2021,449.0,0,0,0,0,0,0,0,0,...,0,0,4.594037e+09,1.767747e+11,38.479161,41.530246,1.125259e+11,6.424882e+10,1.800630e+09,2.793408e+09
261,2021,625.0,0,0,0,0,0,0,0,0,...,0,0,1.438952e+08,1.508488e+11,1048.324019,1107.139880,1.429809e+11,7.867905e+09,7.157256e+07,7.232269e+07
262,2021,629.0,0,0,0,0,0,0,0,0,...,0,0,1.627067e+08,4.950757e+10,304.274861,304.806118,4.585181e+10,3.655754e+09,1.164413e+08,4.626545e+07


In [22]:
# sums

In [23]:
yakuji=pd.read_csv("../merged/len3_ship_agg_drop_never.csv",encoding="shift-jis",index_col=0)
yakuji=yakuji[["year","id","rx_dom","otc_dom","stock_rx","stock_otc","stock_self"]]
aggregated_df=aggregated_df.merge(yakuji,left_on=["year","薬効分類"],right_on=["year","id"],how="left")
aggregated_df.head()

,year,薬効分類,elasped_m6,elasped_m5,elasped_m4,elasped_m3,elasped_m2,elasped_m1,elasped_0,elasped_1,...,revenue0,revenue1,総計0,総計1,id,rx_dom,otc_dom,stock_rx,stock_otc,stock_self
0,2014,114.0,0,0,0,0,0,0,0,0,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
1,2014,121.0,0,0,0,0,0,0,0,0,...,3.016000e+08,1.703117e+07,4.539382e+07,3.006345e+06,121,12343814,0.0,1340818.0,0.0,0.0
2,2014,123.0,0,0,0,0,0,0,0,0,...,1.903159e+09,1.306591e+08,1.390288e+08,2.061612e+07,123,2463354,0.0,181986.0,0.0,0.0
3,2014,124.0,0,0,0,0,0,0,0,0,...,7.625584e+09,2.106043e+09,4.668874e+08,3.620207e+08,124,11742829,310348.0,1248754.0,4765.0,0.0
4,2014,131.0,0,0,0,0,0,0,0,0,...,1.192776e+11,1.043834e+10,7.030695e+08,1.114880e+08,131,286216591,59438256.0,24420274.0,13291858.0,0.0


In [24]:
merged_data=pd.merge(data,aggregated_df,on=["year","薬効分類"],how="left")

In [25]:
merged_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,revenue0,revenue1,総計0,総計1,id_y,rx_dom,otc_dom,stock_rx,stock_otc,stock_self
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,1.036001e+11,5.899116e+09,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0


connect generic usage

In [26]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage

,Year,Percentage
0,2007,34.90
1,2008,35.35
2,2009,35.80
3,2010,37.85
4,2011,39.90
5,2012,43.40
6,2013,46.90
7,2014,51.55
8,2015,56.20
9,2016,61.00


In [27]:
merged_df=merged_data.merge(dusage,left_on="year",right_on="Year",how="left")
merged_df.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,総計0,総計1,id_y,rx_dom,otc_dom,stock_rx,stock_otc,stock_self,Year,Percentage
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,2.621601e+09,8.099551e+08,114,140580346,48217334.0,32986902.0,4489227.0,0.0,2014,51.55


In [28]:
id_dummies=pd.get_dummies(merged_df["薬効分類"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
# share of the product
df_with_dummies["q_share"]=df_with_dummies["総計"]/df_with_dummies["sum_quantity"]
df_with_dummies["r_share"]=df_with_dummies["revenue"]/(df_with_dummies["sum_revenue"]+df_with_dummies["otc_dom"]*1000)

# aggregated generic share in market i (id len 3)
df_with_dummies["generic_share_q"]=df_with_dummies["総計1"]/df_with_dummies["sum_quantity"]
df_with_dummies["generic_share_r"]=df_with_dummies["revenue1"]/df_with_dummies["sum_revenue"]

df_with_dummies.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0.000085,0.000036,0.236031,0.053874
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0.000038,0.000016,0.236031,0.053874
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0.000031,0.000013,0.236031,0.053874
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0.000192,0.000081,0.236031,0.053874
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0.000122,0.000051,0.236031,0.053874


In [29]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
# dummy ver
df_with_dummies.to_csv("../merged/len3_ndb_agg_blp_DropNever.csv",encoding="shift-jis")
# # count ver
# df_with_dummies.to_csv("../merged/len4_ndb_sum_agg_blp_DropNever.csv",encoding="shift-jis")

In [30]:
df_with_dummies.shape

(29653, 117)